# Pipeline in ML 

Pipeline in ml(machine learning) is a technique used to preprocess data,training the model and evaluation within few lines of code. This is important for simplicity and easiness in ml models deployment. 

In [37]:
# import neccessary libraries for demonstration of pipelining 
import pandas as pd 
import seaborn as sb
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score,precision_score 

In [38]:
# load the titanic dataset 
df = sb.load_dataset('titanic') 
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [39]:
# Selecting input features and target variable 
X = df[['pclass','sex','age','fare','embarked']] 
Y = df['survived'] 
# Splitting the data into train and test sets 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42) 

In [40]:
# Defining the column transformer for imputing missing values 
numerical_inputs = ['age','fare'] 
categorical_inputs = ['pclass','sex','embarked']
numeric_transformer = Pipeline(steps=[
    ('imputer',KNNImputer())
])

categories_transformer = Pipeline(steps=[(
    'imputer',SimpleImputer(strategy='most_frequent')   ),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[(
    'num',numeric_transformer,numerical_inputs),
    ('cat',categories_transformer,categorical_inputs)
])

In [41]:
# Creating a pipeline with the preprocessor and RandomForestClassifier 
pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',RandomForestClassifier(random_state=42))
])
# Fitting the pipeline on the training data 
pipeline.fit(X_train,Y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pclass', 'sex',
                                                   'embarked'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [42]:
# checking the accuracy and precision on the test data
y_pred = pipeline.predict(X_test) 
print("Accuracy:",accuracy_score(Y_test,y_pred))
print("Precision:",precision_score(Y_test,y_pred))

Accuracy: 0.8044692737430168
Precision: 0.7671232876712328
